# Snowpark Hugging Face
Let's try out Hugging Face libraries

The process has improved for Transformers and Sentence Transformers, there is native support for Snowpark. 


### 1. Setup Environment

In [1]:
# Snowflake connector
from snowflake import connector
#from snowflake.ml.utils import connection_params

# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import Variant
from snowflake.snowpark.version import VERSION

# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [2]:
with open('creds.json') as f:
    data = json.load(f)
    USERNAME = data['user']
    PASSWORD = data['password']
    SF_ACCOUNT = data['account']
    SF_WH = data['warehouse']

CONNECTION_PARAMETERS = {
   "account": SF_ACCOUNT,
   "user": USERNAME,
   "password": PASSWORD,
}

session = Session.builder.configs(CONNECTION_PARAMETERS).create()

Let's grab the FinBERT text classification model, use it, save it to the registry, and then run prediction jobs. You can also add the parameter `token` to add your Hugging Face token for models that are restricted.

In [3]:
import transformers

finbert_model = transformers.pipeline(
    task="text-classification",
    model="ProsusAI/finbert",
    top_k=2,
)

/Users/rajishah/anaconda3/envs/working38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
finbert_model.predict("I am bullish on $AAPL")

[[{'label': 'neutral', 'score': 0.7302333116531372},
  {'label': 'positive', 'score': 0.18303203582763672}]]

In [ ]:
from snowflake.ml.registry import Registry

reg = Registry(session=session, database_name="RAJIV", schema_name="PUBLIC")

# Log the model
mv = reg.log_model(
    finbert_model,
    model_name="finbert",
    version_name="v1",
    conda_dependencies=["transformers==4.32.1","tokenizers==0.13.3"],
)


In [8]:
mv = reg.get_model("finbert").version("v1")

In [9]:
import pandas as pd 
# Use the model
mv.run(pd.DataFrame(
        [
            ["I have a problem with my Snowflake that needs to be resolved asap!!", ""],
            ["I would like to have udon for today's dinner.", ""],
        ]
    )
)


,outputs
0,"[{""label"": ""negative"", ""score"": 0.810623705387..."
1,"[{""label"": ""neutral"", ""score"": 0.9263970851898..."
